In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()
data = spark.read.csv('/FileStore/tables/Telecom_Churn_Prediction.csv',inferSchema=True,header=True)
data.printSchema()

root
-- customerID: string (nullable = true)
-- gender: string (nullable = true)
-- SeniorCitizen: integer (nullable = true)
-- Partner: string (nullable = true)
-- Dependents: string (nullable = true)
-- tenure: integer (nullable = true)
-- PhoneService: string (nullable = true)
-- MultipleLines: string (nullable = true)
-- InternetService: string (nullable = true)
-- OnlineSecurity: string (nullable = true)
-- OnlineBackup: string (nullable = true)
-- DeviceProtection: string (nullable = true)
-- TechSupport: string (nullable = true)
-- StreamingTV: string (nullable = true)
-- StreamingMovies: string (nullable = true)
-- Contract: string (nullable = true)
-- PaperlessBilling: string (nullable = true)
-- PaymentMethod: string (nullable = true)
-- MonthlyCharges: double (nullable = true)
-- TotalCharges: string (nullable = true)
-- Churn: string (nullable = true)

In [2]:
from pyspark.sql.functions import *
data = data.withColumn('Y', when(data['Churn'] == 'No', 0).otherwise(1))
data = data.drop('customerID')
data = data.drop('TotalCharges')
data = data.drop('Churn')

In [3]:
temp = data.groupBy('gender').agg({"Y": "avg"})
temp = temp.withColumn("ln(Y)", log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+------+-------------------+-------------------+
gender| avg(Y)| ln(Y)|
+------+-------------------+-------------------+
Female|0.26920871559633025|-0.9986409251678525|
 Male| 0.2616033755274262|-1.0376515888784223|
+------+-------------------+-------------------+

In [4]:
#Gender not considered
data = data.drop('gender')

In [5]:
temp = data.groupBy('SeniorCitizen').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+-------------+-------------------+-------------------+
SeniorCitizen| avg(Y)| ln(Y)|
+-------------+-------------------+-------------------+
 1| 0.4168126094570928|-0.3358718163089693|
 0|0.23606168446026096|-1.1743939013798612|
+-------------+-------------------+-------------------+

In [6]:
data = data.withColumn('SeniorCitizen_desc', when(data['SeniorCitizen'] == 0, 'a. No').otherwise('b. Yes'))
data = data.withColumn('SeniorCitizen_val', when(data['SeniorCitizen'] == 0, -1.1744).otherwise(-0.3359))

In [7]:
data = data.drop('SeniorCitizen')
temp = data.groupBy('SeniorCitizen_desc').agg(avg("SeniorCitizen_val"),count("SeniorCitizen_val"))
temp.show()

+------------------+----------------------+------------------------+
SeniorCitizen_desc|avg(SeniorCitizen_val)|count(SeniorCitizen_val)|
+------------------+----------------------+------------------------+
 a. No| -1.1743999999999284| 5901|
 b. Yes| -0.33589999999999615| 1142|
+------------------+----------------------+------------------------+

In [8]:
temp = data.groupBy('Partner','Dependents').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+-------+----------+-------------------+-------------------+
Partner|Dependents| avg(Y)| ln(Y)|
+-------+----------+-------------------+-------------------+
 Yes| Yes| 0.1423670668953688|-1.7957674906255938|
 No| No|0.34237804878048783|-0.6527146916507132|
 Yes| No| 0.2540834845735027|-1.0769507918869303|
 No| Yes|0.21329639889196675|-1.3051688163075224|
+-------+----------+-------------------+-------------------+

In [9]:
data = data.withColumn('ParDep_desc', when((data['Partner'] == 'Yes') & (data['Dependents'] == 'Yes'), 'a. Y Y')
                       .when((data['Partner'] == 'No') & (data['Dependents'] == 'Yes'), 'b. N Y')
                       .when((data['Partner'] == 'Yes') & (data['Dependents'] == 'No'), 'c. Y N')
                       .when((data['Partner'] == 'No') & (data['Dependents'] == 'No'), 'd. N N'))
data = data.withColumn('ParDep_val', when((data['Partner'] == 'Yes') & (data['Dependents'] == 'Yes'), -1.7958)
                       .when((data['Partner'] == 'No') & (data['Dependents'] == 'Yes'), -1.3052)
                       .when((data['Partner'] == 'Yes') & (data['Dependents'] == 'No'), -1.0770)
                       .when((data['Partner'] == 'No') & (data['Dependents'] == 'No'), -0.6527))

In [10]:
data = data.drop('Partner')
data = data.drop('Dependents')
temp = data.groupBy('ParDep_desc').agg(avg("ParDep_val"),count("ParDep_val"))
temp.show()

+-----------+-------------------+-----------------+
ParDep_desc| avg(ParDep_val)|count(ParDep_val)|
+-----------+-------------------+-----------------+
 b. N Y|-1.3052000000000095| 361|
 d. N N|-0.6527000000000436| 3280|
 c. Y N|-1.0769999999999984| 1653|
 a. Y Y|-1.7957999999999847| 1749|
+-----------+-------------------+-----------------+

In [11]:
data = data.withColumn('tenure', when(data['tenure'] == 0, 72).otherwise(data['tenure']))
temp = data.groupBy('tenure').agg({"Y": "avg"})
temp = temp.withColumn("ln(Y)", log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+------+-------------------+--------------------+
tenure| avg(Y)| ln(Y)|
+------+-------------------+--------------------+
 31|0.24615384615384617| -1.1192315758708453|
 65|0.11842105263157894| -2.0074680420547466|
 53| 0.2| -1.3862943611198906|
 34|0.18461538461538463| -1.4853852637641216|
 28|0.21052631578947367| -1.3217558399823195|
 27|0.18055555555555555| -1.5125880864441827|
 26| 0.189873417721519| -1.4508328822574617|
 44|0.11764705882352941| -2.0149030205422647|
 12| 0.3247863247863248| -0.7318616927406357|
 22| 0.3| -0.8472978603872036|
 47|0.20588235294117646| -1.349926716949016|
 1| 0.6199021207177814| 0.4891327991547308|
 52| 0.1| -2.197224577336219|
 13| 0.3486238532110092| -0.6250937173149296|
 16| 0.35| -0.6190392084062235|
 6|0.36363636363636365| -0.5596157879354225|
 3| 0.47|-0.12014431184206334|
 20| 0.2535211267605634| -1.0799201556559572|
 40| 0.203125| -1.3668762752627892|
 57|0.12307692307692308| -1.963609726154714|
+------+-------------------+--------------------+
only showing top 20 rows

In [12]:
data = data.withColumn('tenure_val', -0.0341*data['tenure'] - 0.1588)

In [13]:
data = data.drop('tenure')
data.describe('tenure_val').show()

+-------+-------------------+
summary| tenure_val|
+-------+-------------------+
 count| 7043|
 mean|-1.2664907851767524|
 stddev| 0.8380360113593713|
 min| -2.614|
 max| -0.1929|
+-------+-------------------+

In [14]:
temp = data.groupBy('PhoneService').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+------------+-------------------+-------------------+
PhoneService| avg(Y)| ln(Y)|
+------------+-------------------+-------------------+
 No|0.24926686217008798| -1.102526187989246|
 Yes| 0.2670963684955196|-1.0094046979237703|
+------------+-------------------+-------------------+

In [15]:
#Phone Service not considered
data = data.drop('PhoneService')

In [16]:
temp = data.groupBy('MultipleLines').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+----------------+-------------------+-------------------+
 MultipleLines| avg(Y)| ln(Y)|
+----------------+-------------------+-------------------+
No phone service|0.24926686217008798| -1.102526187989246|
 No| 0.2504424778761062|-1.0962537970088166|
 Yes| 0.286098956580276|-0.9144066050803202|
+----------------+-------------------+-------------------+

In [17]:
# Multiple Lines not considered
data = data.drop('MultipleLines')

In [18]:
temp = data.groupBy('InternetService').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+---------------+-------------------+-------------------+
InternetService| avg(Y)| ln(Y)|
+---------------+-------------------+-------------------+
 Fiber optic| 0.4189276485788114|-0.3271770496340892|
 No|0.07404980340760157| -2.526082563972187|
 DSL| 0.1895910780669145|-1.4526694300647631|
+---------------+-------------------+-------------------+

In [19]:
data = data.withColumn('InternetService_desc', when(data['InternetService'] == 'No', 'a. No')
                       .when(data['InternetService'] == 'DSL', 'b. DSL').otherwise('c. Fiber optic'))
data = data.withColumn('InternetService_val', when(data['InternetService'] == 'No', -2.5261)
                       .when(data['InternetService'] == 'DSL', -1.4527).otherwise(-0.3272))

In [20]:
data = data.drop('InternetService')
temp = data.groupBy('InternetService_desc').agg(avg("InternetService_val"),count("InternetService_val"))
temp.show()

+--------------------+------------------------+--------------------------+
InternetService_desc|avg(InternetService_val)|count(InternetService_val)|
+--------------------+------------------------+--------------------------+
 a. No| -2.526100000000036| 1526|
 b. DSL| -1.4526999999999466| 2421|
 c. Fiber optic| -0.32719999999997634| 3096|
+--------------------+------------------------+--------------------------+

In [21]:
temp = data.groupBy('OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+
 OnlineSecurity| OnlineBackup| DeviceProtection| TechSupport| avg(Y)| ln(Y)|
+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+
 Yes| No| No| Yes|0.15568862275449102| -1.690663352356686|
 Yes| No| No| No| 0.2987012987012987| -0.8534898306351247|
 No| Yes| No| No| 0.416135881104034|-0.33865643843605403|
 Yes| Yes| No| Yes| 0.1004566210045662| -2.192161275379673|
 Yes| Yes| No| No|0.19626168224299065| -1.4098248585300848|
 No| Yes| Yes| Yes| 0.15234375| -1.7163357074108132|
 Yes| No| Yes| No|0.18285714285714286| -1.4971087274601806|
 No| No| Yes| Yes| 0.2375| -1.1664348850068706|
 No| No| No| Yes|0.30991735537190085| -0.8005056988804446|
No internet service|No internet service|No internet service|No internet service|0.07404980340760157| -2.526082563972187|
 No| Yes| Yes| No|0.34959349593495936| -0.620826518980319|
 No| No| No| No| 0.5666929755327546| 0.2683711275386936|
 No| No| Yes| No| 0.4641255605381166|-0.14374475866614161|
 Yes| Yes| Yes| No|0.13452914798206278| -1.8614928072427301|
 Yes| Yes| Yes| Yes|0.05319148936170213| -2.8791984572980396|
 No| Yes| No| Yes| 0.1932367149758454| -1.4291143583028187|
 Yes| No| Yes| Yes|0.10699588477366255| -2.1218008155189776|
+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+

In [22]:
data = data.withColumn('OODT_desc', when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), 'i')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), 'g')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), 'h')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), 'f')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), 'h')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), 'e')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), 'g')
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), 'd')
                       .when((data['OnlineSecurity'] == 'No internet service') & (data['OnlineBackup'] == 'No internet service') & (data['DeviceProtection'] == 'No internet service') & (data['TechSupport'] == 'No internet service'), 'b')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), 'g')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), 'd')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), 'e')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), 'c')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), 'e')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), 'c')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), 'd')
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), 'a'))

data = data.withColumn('OODT_val', when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), 0.2684)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), -0.7461)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), -0.2439)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), -1.1664)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), -0.2439)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), -1.4422)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), -0.7461)
                       .when((data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), -1.7598)
                       .when((data['OnlineSecurity'] == 'No internet service') & (data['OnlineBackup'] == 'No internet service') & (data['DeviceProtection'] == 'No internet service') & (data['TechSupport'] == 'No internet service'), -2.5261)
.when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), -0.7461)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), -1.7598)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), -1.4422)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'No') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), -2.1552)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'No'), -1.4422)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'No') & (data['TechSupport'] == 'Yes'), -2.1552)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'No'), -1.7598)
                       .when((data['OnlineSecurity'] == 'Yes') & (data['OnlineBackup'] == 'Yes') & (data['DeviceProtection'] == 'Yes') & (data['TechSupport'] == 'Yes'), -2.8792))

In [23]:
data = data.drop('OnlineSecurity')
data = data.drop('OnlineBackup')
data = data.drop('DeviceProtection')
data = data.drop('TechSupport')
temp = data.groupBy('OODT_desc').agg(avg("OODT_val"),count("OODT_val"))
temp.show()

+---------+--------------------+---------------+
OODT_desc| avg(OODT_val)|count(OODT_val)|
+---------+--------------------+---------------+
 g| -0.7460999999999957| 919|
 f| -1.166400000000005| 240|
 e| -1.4421999999999888| 596|
 h|-0.24389999999999698| 917|
 d| -1.7598000000000202| 646|
 c| -2.1552000000000127| 462|
 i| 0.2684000000000016| 1267|
 b| -2.526100000000036| 1526|
 a| -2.879200000000013| 470|
+---------+--------------------+---------------+

In [24]:
temp = data.groupBy('StreamingTV','StreamingMovies').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+-------------------+-------------------+-------------------+-------------------+
 StreamingTV| StreamingMovies| avg(Y)| ln(Y)|
+-------------------+-------------------+-------------------+-------------------+
 Yes| Yes| 0.2943298969072165|-0.8744466156324385|
 No| No| 0.344400396432111|-0.6437453185501636|
 Yes| No|0.31681877444589307| -0.768430240980494|
No internet service|No internet service|0.07404980340760157| -2.526082563972187|
 No| Yes|0.31186868686868685| -0.791397458035267|
+-------------------+-------------------+-------------------+-------------------+

In [25]:
data = data.withColumn('Streaming_desc', when((data['StreamingTV'] == 'No internet service') & (data['StreamingMovies'] == 'No internet service'), 'a. No').otherwise('b. Yes'))
data = data.withColumn('Streaming_val', when((data['StreamingTV'] == 'No internet service') & (data['StreamingMovies'] == 'No internet service'), -2.5261).otherwise(-0.7634))

In [26]:
data = data.drop('StreamingTV')
data = data.drop('StreamingMovies')
temp = data.groupBy('Streaming_desc').agg(avg("Streaming_val"),count("Streaming_val"))
temp.show()

+--------------+-------------------+--------------------+
Streaming_desc| avg(Streaming_val)|count(Streaming_val)|
+--------------+-------------------+--------------------+
 a. No| -2.526100000000036| 1526|
 b. Yes|-0.7633999999999236| 5517|
+--------------+-------------------+--------------------+

In [27]:
temp = data.groupBy('Contract').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+--------------+-------------------+--------------------+
 Contract| avg(Y)| ln(Y)|
+--------------+-------------------+--------------------+
Month-to-month| 0.4270967741935484|-0.29370618705516177|
 One year|0.11269517990495587| -2.063501925267679|
 Two year|0.02831858407079646| -3.5355097192697493|
+--------------+-------------------+--------------------+

In [28]:
data = data.withColumn('Contract_desc', when(data['Contract'] == 'Month-to-month', 'b. Month').otherwise('b. Year'))
data = data.withColumn('Contract_val', when(data['Contract'] == 'Month-to-month', -0.2937).otherwise(-2.8511))

In [29]:
data = data.drop('Contract')
temp = data.groupBy('Contract_desc').agg(avg("Contract_val"),count("Contract_val"))
temp.show()

+-------------+--------------------+-------------------+
Contract_desc| avg(Contract_val)|count(Contract_val)|
+-------------+--------------------+-------------------+
 b. Month|-0.29369999999996993| 3875|
 b. Year| -2.8510999999999| 3168|
+-------------+--------------------+-------------------+

In [30]:
temp = data.groupBy('PaperlessBilling','PaymentMethod').agg({"Y": "avg"})
temp = temp.withColumn('ln(Y)', log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+----------------+--------------------+-------------------+--------------------+
PaperlessBilling| PaymentMethod| avg(Y)| ln(Y)|
+----------------+--------------------+-------------------+--------------------+
 Yes|Bank transfer (au...|0.20875420875420875| -1.3324511290990675|
 No|Bank transfer (au...|0.11026033690658499| -2.088084637835856|
 No| Electronic check| 0.3274478330658106| -0.7197509260779213|
 No|Credit card (auto...| 0.1| -2.197224577336219|
 Yes| Mailed check| 0.2728658536585366| -0.980130685047586|
 Yes| Electronic check|0.49770378874856486|-0.00918490957707...|
 Yes|Credit card (auto...|0.19047619047619047| -1.4469189829363254|
 No| Mailed check|0.13493723849372385| -1.8579922906620192|
+----------------+--------------------+-------------------+--------------------+

In [31]:
data = data.withColumn('PBPM_desc', when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Bank transfer (automatic)'), 'a')
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Credit card (automatic)'), 'a')
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Electronic check'), 'c')
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Mailed check'), 'a')
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Bank transfer (automatic)'), 'b')
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Credit card (automatic)'), 'b')
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Electronic check'), 'd')
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Mailed check'), 'c'))

data = data.withColumn('PBPM_val', when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Bank transfer (automatic)'), -2.0213)
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Credit card (automatic)'), -2.0213)
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Electronic check'), -0.8533)
                       .when((data['PaperlessBilling'] == 'No') & (data['PaymentMethod'] == 'Mailed check'), -2.0213)
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Bank transfer (automatic)'), -1.3894)
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Credit card (automatic)'), -1.3894)
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Electronic check'), -0.0092)
                       .when((data['PaperlessBilling'] == 'Yes') & (data['PaymentMethod'] == 'Mailed check'), -0.8533))

In [32]:
data = data.drop('PaperlessBilling')
data = data.drop('PaymentMethod')
temp = data.groupBy('PBPM_desc').agg(avg("PBPM_val"),count("PBPM_val"))
temp.show()

+---------+--------------------+---------------+
PBPM_desc| avg(PBPM_val)|count(PBPM_val)|
+---------+--------------------+---------------+
 d|-0.00919999999999...| 1742|
 c| -0.8532999999999916| 1279|
 b| -1.3894000000000204| 1773|
 a| -2.021300000000005| 2249|
+---------+--------------------+---------------+

In [33]:
temp = data.groupBy('MonthlyCharges').agg({"Y": "avg"})
temp = temp.withColumn("ln(Y)", log(temp['avg(Y)']/(1-temp['avg(Y)'])))
temp.show()

+--------------+-------------------+-------------------+
MonthlyCharges| avg(Y)| ln(Y)|
+--------------+-------------------+-------------------+
 74.5| 0.6666666666666666| 0.6931471805599452|
 78.75|0.14285714285714285| -1.791759469228055|
 64.2| 0.0| null|
 98.3| 0.6666666666666666| 0.6931471805599452|
 56.8| 0.0| null|
 49.8| 0.0| null|
 76.4| 0.6| 0.4054651081081642|
 61.95| 0.0| null|
 69.8|0.42857142857142855|-0.2876820724517809|
 96.35| 0.0| null|
 74.85|0.14285714285714285| -1.791759469228055|
 105.55| 0.2|-1.3862943611198906|
 63.95| 0.0| null|
 109.75| 0.2|-1.3862943611198906|
 110.7| 0.0| null|
 78.9| 0.2857142857142857|-0.9162907318741551|
 82.65| 0.0| null|
 85.65| 0.5| 0.0|
 53.3| 0.0| null|
 113.15| 0.3333333333333333|-0.6931471805599454|
+--------------+-------------------+-------------------+
only showing top 20 rows

In [34]:
data = data.withColumn('MonthlyCharges_desc', when((data['MonthlyCharges'] < 25) | (data['MonthlyCharges'] > 115), 'a').otherwise('b'))
data = data.withColumn('MonthlyCharges_val', when((data['MonthlyCharges'] < 25) | (data['MonthlyCharges'] > 115), -2.2958).otherwise(-0.8315))

In [35]:
data = data.drop('MonthlyCharges')
temp = data.groupBy('MonthlyCharges_desc').agg(avg("MonthlyCharges_val"),count("MonthlyCharges_val"))
temp.show()

+-------------------+-----------------------+-------------------------+
MonthlyCharges_desc|avg(MonthlyCharges_val)|count(MonthlyCharges_val)|
+-------------------+-----------------------+-------------------------+
 b| -0.8314999999999322| 5590|
 a| -2.295799999999977| 1453|
+-------------------+-----------------------+-------------------------+

In [36]:
data = data.drop('SeniorCitizen_desc')
data = data.drop('ParDep_desc')
data = data.drop('InternetService_desc')
data = data.drop('OODT_desc')
data = data.drop('Streaming_desc')
data = data.drop('Contract_desc')
data = data.drop('PBPM_desc')
data = data.drop('MonthlyCharges_desc')

data.printSchema()

root
-- Y: integer (nullable = false)
-- SeniorCitizen_val: double (nullable = false)
-- ParDep_val: double (nullable = true)
-- tenure_val: double (nullable = true)
-- InternetService_val: double (nullable = false)
-- OODT_val: double (nullable = true)
-- Streaming_val: double (nullable = false)
-- Contract_val: double (nullable = false)
-- PBPM_val: double (nullable = true)
-- MonthlyCharges_val: double (nullable = false)

In [37]:
from pyspark.mllib.stat import Statistics
import pandas as pd
col_names = data.columns
features = data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names
print(corr_df.to_string())

Y SeniorCitizen_val ParDep_val tenure_val InternetService_val OODT_val Streaming_val Contract_val PBPM_val MonthlyCharges_val
Y 1.000000 0.150889 0.184649 0.354745 0.317659 0.417540 0.227890 0.405103 0.330109 0.200726
SeniorCitizen_val 0.150889 1.000000 0.139724 -0.014543 0.260151 0.206570 0.182742 0.138360 0.195750 0.170973
ParDep_val 0.184649 0.139724 1.000000 0.295838 0.125089 0.236204 0.097706 0.293483 0.160712 0.074626
tenure_val 0.354745 -0.014543 0.295838 1.000000 -0.028399 0.403732 -0.035384 0.650193 0.198156 -0.053097
InternetService_val 0.317659 0.260151 0.125089 -0.028399 1.000000 0.591033 0.817789 0.271365 0.435690 0.718170
OODT_val 0.417540 0.206570 0.236204 0.403732 0.591033 1.000000 0.598575 0.543117 0.434765 0.552463
Streaming_val 0.227890 0.182742 0.097706 -0.035384 0.817789 0.598575 1.000000 0.218639 0.353120 0.881670
Contract_val 0.405103 0.138360 0.293483 0.650193 0.271365 0.543117 0.218639 1.000000 0.352277 0.184393
PBPM_val 0.330109 0.195750 0.160712 0.198156 0.435690 0.434765 0.353120 0.352277 1.000000 0.317495
MonthlyCharges_val 0.200726 0.170973 0.074626 -0.053097 0.718170 0.552463 0.881670 0.184393 0.317495 1.000000

In [38]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['SeniorCitizen_val','ParDep_val','tenure_val','InternetService_val','OODT_val','Streaming_val','Contract_val','PBPM_val','MonthlyCharges_val'],outputCol='features')
output = assembler.transform(data)
final_data = output.select('features','Y')

In [39]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="binomial", link="logit", maxIter=100, regParam=0.0, labelCol='Y')
model = glr.fit(final_data)
summary = model.summary
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("AIC: " + str(summary.aic))

Coefficients: [0.344669430046,0.143027014179,0.830929866579,0.858701994174,0.278806807868,-0.124614045755,0.307063355917,0.377971444455,-0.248229713264]
Intercept: 1.8535002705702428
Coefficient Standard Errors: [0.09901346802446975, 0.0790856751190786, 0.060918541834556125, 0.06780286717643925, 0.04753674679386483, 0.15329328538144693, 0.03958648741025958, 0.045324085817035197, 0.15819964180093768, 0.15221804271973408]
T Values: [3.4810358320232417, 1.808507216552654, 13.640015692355831, 12.664685579438563, 5.865079684075559, -0.8129126167868677, 7.756771969552067, 8.339306521944177, -1.569091500067388, 12.176613477962876]
P Values: [0.0004994786672203322, 0.07052759466917946, 0.0, 0.0, 4.489170590815661e-09, 0.41626816026319413, 8.659739592076221e-15, 0.0, 0.11662662007191082, 0.0]
AIC: 5964.669928316919

In [40]:
data = data.drop('Streaming_val')
data = data.drop('ParDep_val')
data = data.drop('MonthlyCharges_val')
assembler = VectorAssembler(inputCols=['SeniorCitizen_val','tenure_val','InternetService_val','OODT_val','Contract_val','PBPM_val'],outputCol='features')
output = assembler.transform(data)
final_data = output.select('features','Y')

In [41]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="binomial", link="logit", maxIter=100, regParam=0.0, labelCol='Y')
model = glr.fit(final_data)
summary = model.summary
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("AIC: " + str(summary.aic))

Coefficients: [0.378649034266,0.871536117222,0.741045841076,0.216632047472,0.317819364121,0.379118496302]
Intercept: 2.0002576152357636
Coefficient Standard Errors: [0.09800881216535709, 0.059544310841475656, 0.05648106435564004, 0.04258669990426138, 0.03939821540964977, 0.045166722768569964, 0.12257360872276725]
T Values: [3.863418256994561, 14.636765543267922, 13.120252770209655, 5.086847488970936, 8.066846704008992, 8.393757019832812, 16.318827813578345]
P Values: [0.00011181132009596695, 0.0, 0.0, 3.6406436199065695e-07, 6.661338147750939e-16, 0.0, 0.0]
AIC: 5974.2213578864585

In [42]:
col_names = data.columns
features = data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names
print(corr_df.to_string())

Y SeniorCitizen_val tenure_val InternetService_val OODT_val Contract_val PBPM_val
Y 1.000000 0.150889 0.354745 0.317659 0.417540 0.405103 0.330109
SeniorCitizen_val 0.150889 1.000000 -0.014543 0.260151 0.206570 0.138360 0.195750
tenure_val 0.354745 -0.014543 1.000000 -0.028399 0.403732 0.650193 0.198156
InternetService_val 0.317659 0.260151 -0.028399 1.000000 0.591033 0.271365 0.435690
OODT_val 0.417540 0.206570 0.403732 0.591033 1.000000 0.543117 0.434765
Contract_val 0.405103 0.138360 0.650193 0.271365 0.543117 1.000000 0.352277
PBPM_val 0.330109 0.195750 0.198156 0.435690 0.434765 0.352277 1.000000

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Y')
results = model.transform(final_data)
AUC = my_eval.evaluate(results)
AUC

Out[ 43 ]: 0.8399691278551867